In [36]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [37]:
from model_arch.run_train import *
from utils.step_sample import create_named_schedule_sampler
from model_arch.train import TrainLoop
from utils.data import load_data_text
from model_arch.tokenizer import load_tokenizer, load_model_emb
from model_arch.sampling import sampling

from transformers import AutoTokenizer, PreTrainedTokenizerFast, BertTokenizerFast, set_seed
import json, torch
from utils import dist_util
from functools import partial
import pickle
import random
from datetime import datetime

In [38]:
dist_util.clear_cache()

In [43]:
lr=0.0001
batch_size=30
val_batch_size=30
microbatch=10
epochs=30_000
eval_interval=10
ema_rate='0.9999' 
schedule_sampler='uniform'
diffusion_steps=2000
noise_schedule='sqrt'
vocab='custom'
use_plm_init='no' # embedding in transformer
vocab_size=0
config_name='bert-base-uncased'
seq_len=128
hidden_t_dim=128
hidden_dim=128
dropout=0.1
seed=10275679
weight_decay=0.0
predict_xstart=True
rescale_timesteps=True
emb_scale_factor=1.0

In [44]:
regular_data_dir='data'
data_player_dir='data/with_player'

# set the data directory
data_dir=regular_data_dir

In [45]:
set_seed(seed)

In [46]:
tokenizer = load_tokenizer('shakespeare', config_name)

shakespeare


In [47]:
model_weight, tokenizer = load_model_emb(hidden_dim, tokenizer)

In [48]:
model_weight

Embedding(30268, 128)

In [49]:
## very very important to set this!!!!!
vocab_size = tokenizer.vocab_size
vocab_size

30268

In [50]:
data = load_data_text(
        batch_size=batch_size,
        seq_len=seq_len,
        data_dir=data_dir,
        loaded_vocab=tokenizer,
        model_emb=model_weight # use model's weights as init
    )

val = load_data_text(
        batch_size=val_batch_size,
        seq_len=seq_len,
        data_dir=data_dir,
        loaded_vocab=tokenizer,
        split='valid',
        model_emb=model_weight, # use model's weights as init
    )

############################## 
Loading text data...
############################## 
Loading dataset from data...
### Loading form the TRAIN set...
### Data samples...
 ['this is the very false gallop of verses why do you infect yourself with them?', 'and fair men call for grace. aaron will have his soul black like his face.'] ['peace, you dull fool! i found them on a tree.', 'o, here i lift this one hand up to heaven, and bow this feeble ruin to the earth if any power pities wretched tears, to that i call! what, wilt thou kneel with me? do, then, dear heart,']
RAM used: 5815.46 MB
This is raw_datasets:  Dataset({
    features: ['src', 'trg'],
    num_rows: 32531
})
RAM used: 5842.03 MB


Running tokenizer on dataset (num_proc=4):   0%|          | 0/32531 [00:00<?, ? examples/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f054559c3a0>
Traceback (most recent call last):
  File "/home/xykong/.local/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1479, in __del__
    self._shutdown_workers()
  File "/home/xykong/.local/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1462, in _shutdown_workers
    if w.is_alive():
  File "/opt/conda/lib/python3.9/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f054559c3a0>
Traceback (most recent call last):
  File "/home/xykong/.local/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1479, in __del__
    self._shutdown_workers()
  File "/home/xykong/.local/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1462, in _shutdown_workers
 

### tokenized_datasets Dataset({
    features: ['input_id_x', 'input_id_y'],
    num_rows: 32531
})
### tokenized_datasets...example [2, 138, 121, 78, 476, 880, 9871, 94, 5833, 329, 144, 89, 2960, 955, 131, 266, 22, 3]
RAM used: 5850.93 MB


merge and mask:   0%|          | 0/32531 [00:00<?, ? examples/s]

...

### tokenized_datasets Dataset({
    features: ['input_id_x', 'input_id_y'],
    num_rows: 8123
})
### tokenized_datasets...example [2, 714, 37, 9, 749, 243, 23, 550, 85, 317, 31, 241, 125, 100, 208, 10, 6206, 10, 136, 469, 23187, 180, 646, 37, 9, 579, 3437, 10, 85, 1044, 549, 31, 9, 78, 5789, 237, 571, 295, 37, 9, 78, 14784, 12, 3]
RAM used: 5843.07 MB


merge and mask:   0%|          | 0/8123 [00:00<?, ? examples/s]

RAM used: 5846.93 MB


padding:   0%|          | 0/8123 [00:00<?, ? examples/s]

Dataset({
    features: ['input_id_x', 'input_id_y', 'input_ids', 'input_mask'],
    num_rows: 8123
}) padded dataset
RAM used: 5861.21 MB
RAM used: 5861.41 MB


In [51]:
model, diffusion = create_model_and_diffusion(
                        hidden_t_dim,
                        hidden_dim,
                        vocab_size,
                        config_name,
                        use_plm_init,
                        dropout,
                        diffusion_steps,
                        noise_schedule,
                        predict_xstart,
                        rescale_timesteps,
                    )

In [52]:
pytorch_total_params = sum(p.numel() for p in model.parameters())
pytorch_total_params

91192508

In [53]:
from torch import nn
from transformers import BertConfig

config = BertConfig.from_pretrained("bert-base-uncased")

for layer in model.input_transformers.layer[-1:]:
    for module in layer.modules():
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)

In [54]:
schedule_sampler = create_named_schedule_sampler('uniform', diffusion)

model.to(dist_util.dev())

train_loss, val_loss = TrainLoop(
                            model=model,
                            diffusion=diffusion,
                            data=data,
                            batch_size=batch_size,
                            microbatch=microbatch,
                            lr=lr,
                            ema_rate=ema_rate,
                            schedule_sampler=schedule_sampler,
                            weight_decay=weight_decay,
                            epochs=epochs,
                            eval_data=val,
                            eval_interval=eval_interval,
                            warm_up_steps=500,
                            use_llrd=True,
                            llrd_rate=0.9
                        ).run_loop()

dt = datetime.now().strftime("%m%d")
pickle.dump(model, open(f"models/{dt}/final_model_df{diffusion_steps}.pkl", 'wb'))



======== Using Layer-wise Learning Rate Decay with AdamW ========


name: word_embedding.weight, lr: 0.0001
name: lm_head.bias, lr: 0.0001
name: time_embed.0.weight, lr: 0.0001
name: time_embed.0.bias, lr: 0.0001
name: time_embed.2.weight, lr: 0.0001
name: time_embed.2.bias, lr: 0.0001
name: input_up_proj.0.weight, lr: 0.0001
name: input_up_proj.0.bias, lr: 0.0001
name: input_up_proj.2.weight, lr: 0.0001
name: input_up_proj.2.bias, lr: 0.0001
name: input_transformers.layer.0.attention.self.query.weight, lr: 0.00011111111111111112
name: input_transformers.layer.0.attention.self.query.bias, lr: 0.00011111111111111112
name: input_transformers.layer.0.attention.self.key.weight, lr: 0.00011111111111111112
name: input_transformers.layer.0.attention.self.key.bias, lr: 0.00011111111111111112
name: input_transformers.layer.0.attention.self.value.weight, lr: 0.00011111111111111112
name: input_transformers.layer.0.attention.self.value.bias, lr: 0.00011111111111111112
name: input_transformers.la

Epoch 0/30000 Training Loss: 0.9819819331169128
Epoch 0/30000 Validation Loss: 0.9883999228477478
Cleared directory to save new best model.
============>Saving current best model with min_val_loss=0.9883999228477478<=============
Epoch 1/30000 Training Loss: 0.9722307324409485
Epoch 2/30000 Training Loss: 0.983376681804657
Epoch 3/30000 Training Loss: 0.9820019602775574
Epoch 4/30000 Training Loss: 0.9750997424125671
Epoch 5/30000 Training Loss: 0.9565172791481018
Epoch 6/30000 Training Loss: 0.9400200843811035
Epoch 7/30000 Training Loss: 0.9240044951438904
Epoch 8/30000 Training Loss: 0.9050349593162537
Epoch 9/30000 Training Loss: 0.8984633088111877
Epoch 10/30000 Training Loss: 0.894487202167511
Epoch 10/30000 Validation Loss: 0.8452692031860352
Cleared directory to save new best model.
============>Saving current best model with min_val_loss=0.8452692031860352<=============
Epoch 11/30000 Training Loss: 0.8517823815345764
Epoch 12/30000 Training Loss: 0.8533902168273926
Epoch 13/3

Epoch 129/30000 Training Loss: 0.3599205017089844
Epoch 130/30000 Training Loss: 0.32155656814575195
Epoch 130/30000 Validation Loss: 0.3844555914402008
Epoch 131/30000 Training Loss: 0.4106590449810028
Epoch 132/30000 Training Loss: 0.3767671585083008
Epoch 133/30000 Training Loss: 0.3866671025753021
Epoch 134/30000 Training Loss: 0.3607451915740967
Epoch 135/30000 Training Loss: 0.3676261007785797
Epoch 136/30000 Training Loss: 0.3507063686847687
Epoch 137/30000 Training Loss: 0.40308013558387756
Epoch 138/30000 Training Loss: 0.34429264068603516
Epoch 139/30000 Training Loss: 0.3306223154067993
Epoch 140/30000 Training Loss: 0.4115898609161377
Epoch 140/30000 Validation Loss: 0.3504294455051422
Epoch 141/30000 Training Loss: 0.35876819491386414
Epoch 142/30000 Training Loss: 0.3513745367527008
Epoch 143/30000 Training Loss: 0.3308451473712921
Epoch 144/30000 Training Loss: 0.29331114888191223
Epoch 145/30000 Training Loss: 0.34524571895599365
Epoch 146/30000 Training Loss: 0.2978759

Epoch 260/30000 Training Loss: 0.20065145194530487
Epoch 260/30000 Validation Loss: 0.20591551065444946
Epoch 261/30000 Training Loss: 0.2211679369211197
Epoch 262/30000 Training Loss: 0.23936469852924347
Epoch 263/30000 Training Loss: 0.2120223045349121
Epoch 264/30000 Training Loss: 0.21752123534679413
Epoch 265/30000 Training Loss: 0.20072340965270996
Epoch 266/30000 Training Loss: 0.22194546461105347
Epoch 267/30000 Training Loss: 0.21594597399234772
Epoch 268/30000 Training Loss: 0.19564342498779297
Epoch 269/30000 Training Loss: 0.204772487282753
Epoch 270/30000 Training Loss: 0.1884755641222
Epoch 270/30000 Validation Loss: 0.19389593601226807
Cleared directory to save new best model.
============>Saving current best model with min_val_loss=0.19389593601226807<=============
Epoch 271/30000 Training Loss: 0.17813782393932343
Epoch 272/30000 Training Loss: 0.19058942794799805
Epoch 273/30000 Training Loss: 0.19039355218410492
Epoch 274/30000 Training Loss: 0.1867954581975937
Epoch

Epoch 397/30000 Training Loss: 0.17198598384857178
Epoch 398/30000 Training Loss: 0.15589483082294464
Epoch 399/30000 Training Loss: 0.18261097371578217
Epoch 400/30000 Training Loss: 0.14367592334747314
Epoch 400/30000 Validation Loss: 0.142687126994133
Cleared directory to save new best model.
============>Saving current best model with min_val_loss=0.142687126994133<=============
Epoch 401/30000 Training Loss: 0.1920136660337448
Epoch 402/30000 Training Loss: 0.17747925221920013
Epoch 403/30000 Training Loss: 0.19370043277740479
Epoch 404/30000 Training Loss: 0.1553432196378708
Epoch 405/30000 Training Loss: 0.1468883901834488
Epoch 406/30000 Training Loss: 0.1578129529953003
Epoch 407/30000 Training Loss: 0.11395538598299026
Epoch 408/30000 Training Loss: 0.18439966440200806
Epoch 409/30000 Training Loss: 0.13451765477657318
Epoch 410/30000 Training Loss: 0.14785538613796234
Epoch 410/30000 Validation Loss: 0.13887101411819458
Cleared directory to save new best model.
============>

...

Epoch 29412/30000 Training Loss: 0.0587499737739563
Epoch 29413/30000 Training Loss: 0.05361446738243103
Epoch 29414/30000 Training Loss: 0.05656786635518074
Epoch 29415/30000 Training Loss: 0.05260641500353813
Epoch 29416/30000 Training Loss: 0.04727217182517052
Epoch 29417/30000 Training Loss: 0.04567262530326843
Epoch 29418/30000 Training Loss: 0.06717052310705185
Epoch 29419/30000 Training Loss: 0.04424450173974037
Epoch 29420/30000 Training Loss: 0.06342219561338425
Epoch 29420/30000 Validation Loss: 0.05036996677517891
Epoch 29421/30000 Training Loss: 0.03549877181649208
Epoch 29422/30000 Training Loss: 0.06379979103803635
Epoch 29423/30000 Training Loss: 0.040228161960840225
Epoch 29424/30000 Training Loss: 0.0394539050757885
Epoch 29425/30000 Training Loss: 0.04905747249722481
Epoch 29426/30000 Training Loss: 0.06297298520803452
Epoch 29427/30000 Training Loss: 0.05959709361195564
Epoch 29428/30000 Training Loss: 0.05212271586060524
Epoch 29429/30000 Training Loss: 0.0765925124

Epoch 29552/30000 Training Loss: 0.04145347699522972
Epoch 29553/30000 Training Loss: 0.06297723203897476
Epoch 29554/30000 Training Loss: 0.03702203556895256
Epoch 29555/30000 Training Loss: 0.04050073400139809
Epoch 29556/30000 Training Loss: 0.039753492921590805
Epoch 29557/30000 Training Loss: 0.05122438073158264
Epoch 29558/30000 Training Loss: 0.049267735332250595
Epoch 29559/30000 Training Loss: 0.04961884021759033
Epoch 29560/30000 Training Loss: 0.05092024803161621
Epoch 29560/30000 Validation Loss: 0.038804952055215836
Epoch 29561/30000 Training Loss: 0.05768778547644615
Epoch 29562/30000 Training Loss: 0.06281302124261856
Epoch 29563/30000 Training Loss: 0.05527055263519287
Epoch 29564/30000 Training Loss: 0.046862710267305374
Epoch 29565/30000 Training Loss: 0.05278927460312843
Epoch 29566/30000 Training Loss: 0.05898566171526909
Epoch 29567/30000 Training Loss: 0.042891975492239
Epoch 29568/30000 Training Loss: 0.07603278011083603
Epoch 29569/30000 Training Loss: 0.0447078

Epoch 29692/30000 Training Loss: 0.06545606255531311
Epoch 29693/30000 Training Loss: 0.036335188895463943
Epoch 29694/30000 Training Loss: 0.056153398007154465
Epoch 29695/30000 Training Loss: 0.048977404832839966
Epoch 29696/30000 Training Loss: 0.041019633412361145
Epoch 29697/30000 Training Loss: 0.04932806268334389
Epoch 29698/30000 Training Loss: 0.0423416830599308
Epoch 29699/30000 Training Loss: 0.06692443042993546
Epoch 29700/30000 Training Loss: 0.042854756116867065
Epoch 29700/30000 Validation Loss: 0.04003790020942688
Epoch 29701/30000 Training Loss: 0.051561594009399414
Epoch 29702/30000 Training Loss: 0.0503719262778759
Epoch 29703/30000 Training Loss: 0.057412710040807724
Epoch 29704/30000 Training Loss: 0.04797932505607605
Epoch 29705/30000 Training Loss: 0.05630268529057503
Epoch 29706/30000 Training Loss: 0.06335505843162537
Epoch 29707/30000 Training Loss: 0.04352371767163277
Epoch 29708/30000 Training Loss: 0.058419812470674515
Epoch 29709/30000 Training Loss: 0.053

Epoch 29832/30000 Training Loss: 0.036061059683561325
Epoch 29833/30000 Training Loss: 0.06105228140950203
Epoch 29834/30000 Training Loss: 0.054305076599121094
Epoch 29835/30000 Training Loss: 0.048774588853120804
Epoch 29836/30000 Training Loss: 0.04333090782165527
Epoch 29837/30000 Training Loss: 0.050296600908041
Epoch 29838/30000 Training Loss: 0.048807527869939804
Epoch 29839/30000 Training Loss: 0.04391184076666832
Epoch 29840/30000 Training Loss: 0.0562707893550396
Epoch 29840/30000 Validation Loss: 0.05555206909775734
Epoch 29841/30000 Training Loss: 0.0737132728099823
Epoch 29842/30000 Training Loss: 0.044856954365968704
Epoch 29843/30000 Training Loss: 0.05226771533489227
Epoch 29844/30000 Training Loss: 0.04122490808367729
Epoch 29845/30000 Training Loss: 0.05910639092326164
Epoch 29846/30000 Training Loss: 0.055594947189092636
Epoch 29847/30000 Training Loss: 0.061708737164735794
Epoch 29848/30000 Training Loss: 0.04714876040816307
Epoch 29849/30000 Training Loss: 0.051884

Epoch 29972/30000 Training Loss: 0.05167277157306671
Epoch 29973/30000 Training Loss: 0.03930480033159256
Epoch 29974/30000 Training Loss: 0.05879119411110878
Epoch 29975/30000 Training Loss: 0.059162359684705734
Epoch 29976/30000 Training Loss: 0.07243648171424866
Epoch 29977/30000 Training Loss: 0.046790119260549545
Epoch 29978/30000 Training Loss: 0.05353151261806488
Epoch 29979/30000 Training Loss: 0.04625736176967621
Epoch 29980/30000 Training Loss: 0.06120925024151802
Epoch 29980/30000 Validation Loss: 0.048721667379140854
Epoch 29981/30000 Training Loss: 0.05401421710848808
Epoch 29982/30000 Training Loss: 0.047958776354789734
Epoch 29983/30000 Training Loss: 0.076699398458004
Epoch 29984/30000 Training Loss: 0.05788425728678703
Epoch 29985/30000 Training Loss: 0.05088481679558754
Epoch 29986/30000 Training Loss: 0.04553760960698128
Epoch 29987/30000 Training Loss: 0.053281743079423904
Epoch 29988/30000 Training Loss: 0.07540764659643173
Epoch 29989/30000 Training Loss: 0.049881

In [ ]:
# plot(train_loss, val_loss)

In [ ]:
# param_names = []
# for i, (name, param) in enumerate(model.named_parameters()):
#     param_names.append(name)
#     print(f'{i}: {name} {param.requires_grad}')

In [55]:
best_model_fp = f'models/0308/model_best_epoch_23930_min_val_loss_0.02459999918937683.pkl'
with open(best_model_fp, 'rb') as handle:
    best_model = pickle.load(handle)

# Generating sequences

In [90]:
diffusion_2000 = SpacedDiffusion(
    betas=get_named_beta_schedule(noise_schedule, 2000),
    rescale_timesteps=rescale_timesteps,
    predict_xstart=predict_xstart,
)

In [97]:
word_lst_source, word_lst_recover, word_lst_ref, inter_lst_recover = sampling(best_model, 
                                                           diffusion_2000, 
                                                           tokenizer, 
                                                           data_dir=regular_data_dir, 
                                                           batch_size=10, 
                                                           split='test_custom', 
                                                           seq_len=128, 
                                                           show_intermediate_results=False)

############################## 
Loading text data...
############################## 
Loading dataset from data...
### Loading from the custom TEST set...
### Data samples...
 ["my lord, i claim your gift, my due by promise, for which your honour and your faith is pawn'd, the earldom of hereford and the moveables the which you promised i should possess.", 'i am a bastard too, i love bastards i am a bastard begot, bastard instructed, bastard in mind, bastard in valour, in every thing illegitimate. one bear will not bite another,'] ['stanley, look to your wife, if she convey letters to richmond, you shall answer it.', "and wherefore should one bastard? take heed, the quarrel's most ominous to us if the son of a whore fight for a whore, he tempts judgment farewell, bastard."]
RAM used: 6068.92 MB
This is raw_datasets:  Dataset({
    features: ['src', 'trg'],
    num_rows: 4
})
RAM used: 6068.92 MB


Running tokenizer on dataset (num_proc=4):   0%|          | 0/4 [00:00<?, ? examples/s]

### tokenized_datasets Dataset({
    features: ['input_id_x', 'input_id_y'],
    num_rows: 4
})
### tokenized_datasets...example [2, 105, 203, 10, 31, 2385, 135, 1820, 10, 105, 2280, 193, 1823, 10, 115, 260, 135, 487, 85, 135, 676, 121, 3876, 9, 26, 10, 78, 11495, 94, 3787, 85, 78, 11788, 78, 260, 89, 3808, 31, 305, 1872, 12, 3]
RAM used: 6068.98 MB


merge and mask:   0%|          | 0/4 [00:00<?, ? examples/s]

RAM used: 6068.98 MB


padding:   0%|          | 0/4 [00:00<?, ? examples/s]

Dataset({
    features: ['input_id_x', 'input_id_y', 'input_ids', 'input_mask'],
    num_rows: 4
}) padded dataset
RAM used: 6069.00 MB
RAM used: 6069.00 MB
### End of reading iteration...


  0%|          | 0/2000 [00:00<?, ?it/s]

Generating 20 sentences takes 5 minutes

In [98]:
word_lst_source

["[CLS] my lord, i claim your gift, my due by promise, for which your honour and your faith is pawn'd, the earldom of hereford and the moveables the which you promised i should possess. [SEP] [SEP]",
 '[CLS] i am a bastard too, i love bastards i am a bastard begot, bastard instructed, bastard in mind, bastard in valour, in every thing illegitimate. one bear will not bite another, [SEP] [SEP]',
 "[CLS] my lord,'tis the pondering of life's meaning that doth occupy my thoughts most gravely. [SEP] [SEP]",
 '[CLS] call her forth to me [SEP] [SEP]']

In [99]:
word_lst_recover

['[CLS] so your to be him king in? [SEP]',
 '[CLS] by sorry well to be to to, or to the good. [SEP]',
 '[CLS] for your one to a christian. [SEP]',
 '[CLS] how hark! [SEP]']